# 簡単なQ&Aを行うLLMを作成する
このノートブックでは、Hugging FaceのTransformersとPEFTを使用して、簡単なQ&Aを行うLLMを作成します。  
データセットはhugging faceで公開されているdatasetを使用してInstruction Tuningを行います。  
今回使用するLLMは軽量な"llm-jp/llm-jp-3-1.8b"を使用します。  

リンク：  
- データセット：https://huggingface.co/datasets/izumi-lab/llm-japanese-dataset
- モデル：https://huggingface.co/llm-jp/llm-jp-3-1.8b
-  参考：https://qiita.com/m__k/items/173ade78990b7d6a4be4

バージョン情報：  
- python : 3.12.4
- cuda : 12.1
- transformers : 4.44.2
- torch : 2.3.1+cu121
- peft : 0.11.1
- accelerate : 0.32.1
- datasets : 2.20.0


---
### モデルとトークナイザの読み込み
config.yamlを読み込んで指定したモデルとトークナイザをダウンロードします。  
初回実行時はダウンロードに時間がかかることがあります。  
モデルはキャッシュとして保存されるため、必要のないモデルは削除してください。  
モデルの保存先（デフォルト）：
~/.cache/huggingface

In [97]:
import torch
import yaml
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# YAMLファイルの読み込み
with open("./config.yaml", "r") as file:
    config = yaml.safe_load(file)

# モデルの設定
model_config = config["model_config"]
# 生成時の設定
generate_config = config["generate_config"]
# パス設定
paths = config["paths"]
# データセットの設定
dataset_config = config["dataset_config"]

In [93]:
# トークナイザとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_config["model"])
model = AutoModelForCausalLM.from_pretrained(
    model_config["model"], **model_config["model_kwargs"]
)

# streamerを用いるとモデルの推論時に標準出力にストリーミング出力してくれます
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

### モデルの推論
streamerを用いることでストリーミング出力してくれます。    
また、通常の出力はoutput[0]で確認できます。  
小さなモデルなので長文出力の精度は微妙です。  

In [98]:
text = "自然言語処理とは何ですか？"
tokenized_input = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(
        tokenized_input,
        **generate_config,
        streamer=streamer
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




言語処理は、人間の言語を処理する技術です。

言語の処理は、大きく分けて2つあります。

1つ目は、言語の意味を理解する処理です。

例えば、「りんご」という言葉を聞い


In [99]:
print(tokenizer.decode(output[0]))

自然言語処理とは何ですか？

言語処理は、人間の言語を処理する技術です。

言語の処理は、大きく分けて2つあります。

1つ目は、言語の意味を理解する処理です。

例えば、「りんご」という言葉を聞い


## Accelerator
Acceleratorは複数のGPUを用いて分散学習を行う際に使用するツールです。  
分散学習を行わないときには使用しなくても問題ありません。  
参考：https://qiita.com/m__k/items/518ac10399c6c8753763

In [4]:
# from accelerate import Accelerator

# Acceleratorの初期化
# accelerator = Accelerator()

### データセットの準備
まずはhugging faceのdatasetを読み込んで学習データと検証データに分割します。  

In [100]:
# データセットを作成
import datasets

dolly_ja = datasets.load_dataset(dataset_config["name"])


In [101]:
# データセットの中身を確認
dolly_ja['train'][0]

{'output': 'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。',
 'input': 'ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。',
 'index': '0',
 'category': 'closed_qa',
 'instruction': 'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？'}

### トークン化
学習データをLLMに入力するためにトークン化を行います。  
学習データは以下のようなフォーマットにしていますが、利用するモデルによって適切なフォーマットが異なる場合があることに注意してください。    
```
system: あなたは役立つAIアシスタントです。ユーザからの質問に対して回答を行ってください。

user: 自然言語処理とは何ですか？

assistant: 自然言語処理は、コンピュータが人間の言語を理解し、それを利用してコミュニケーションを行うことを可能にする技術です。
```


In [102]:
# データセットの分割
train_valid_split = dolly_ja['train'].train_test_split(test_size=dataset_config["test_size"])
train_dataset = train_valid_split['train']
valid_dataset = train_valid_split['test']

# データセットのトークン化関数
def tokenize_function(example):
    prompts = f"system: あなたは優秀なAIアシスタントです。ユーザからの質問に対して簡潔に回答をしてください。\n\nuser: {example['instruction']}\n\ninput: {example['input']}\n\nassistant: {example['output']}"
    return tokenizer(prompts, padding=False, truncation=True, max_length=model_config["max_length"])

# データセットのトークン化
tokenized_train_dataset = train_dataset.map(tokenize_function)
tokenized_valid_dataset = valid_dataset.map(tokenize_function)

Map: 100%|██████████| 1502/1502 [00:00<00:00, 1575.27 examples/s]


In [103]:
tokenized_train_dataset[0]

{'output': 'アフリカ、アジア、スペイン、ベトナム、中国、ヨーロッパ、北アメリカ',
 'input': '',
 'index': '5746',
 'category': 'classification',
 'instruction': 'これらは国なのか大陸なのか、教えてください',
 'input_ids': [1,
  1598,
  28752,
  39237,
  29282,
  69967,
  11749,
  58023,
  78439,
  64098,
  78486,
  76285,
  99123,
  68068,
  29083,
  75506,
  79087,
  18,
  18,
  1849,
  28752,
  39790,
  29577,
  29282,
  29458,
  56879,
  68660,
  79538,
  71033,
  79833,
  18,
  18,
  2547,
  28752,
  279,
  18,
  18,
  504,
  11151,
  28752,
  41187,
  29046,
  58026,
  29046,
  60691,
  29046,
  63223,
  29046,
  65303,
  29046,
  64132,
  29046,
  29804,
  58168],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


### PEFTを用いたLoRA学習の設定
Loraの設定を行います。  
参考：https://qiita.com/t-hashiguchi/items/9f3b394ca0ae1c7e4d02

In [104]:
from peft import get_peft_model, LoraConfig, PeftModel

model.enable_input_require_grads()
model.gradient_checkpointing_enable(
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
    r=model_config["lora_config"]["lora_r"],
    lora_alpha=model_config["lora_config"]["lora_alpha"],
    lora_dropout=model_config["lora_config"]["lora_dropout"],
)
# ベースモデルをフリーズ
for name, param in model.named_parameters():
    if "lora" not in name and param.ndim == 1:
        param.data = param.data.to(torch.bfloat16)

# モデルにLoRAアダプター適用、更新対象のパラメータ数の確認
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()



trainable params: 10,080,256 || all params: 1,877,694,464 || trainable%: 0.5368


### 学習の開始
hugging faceのtrainerを使用して学習を行います。  
training_args内の細かい設定はLoRAのwebサイトを参考にしてください。  

In [105]:
from transformers import (
    TrainingArguments,
    DataCollatorForLanguageModeling,
    Trainer,
)
from datetime import datetime, timedelta, timezone

# パラメータの保存先を設定
model_name = model_config["model"].split("/")[-1]
JST = timezone(timedelta(hours=+9), "JST")
dt_now = datetime.now(JST)
now_time = dt_now.strftime("%Y%m%d_%H%M%S")
model_dir_path = f"{paths["output_path"]}/{model_name}/{now_time}"

# 学習時のパラメータなどの設定
training_args = TrainingArguments(
    per_device_train_batch_size=model_config['batch_size'],
    per_device_eval_batch_size=model_config['batch_size'],
    learning_rate=model_config['learning_rate'],
    num_train_epochs=model_config['epochs'],
    save_strategy="steps",
    save_steps=model_config['save_steps'],
    evaluation_strategy="steps",
    eval_steps=model_config['logging_steps'],
    logging_strategy="steps",
    logging_steps=model_config['logging_steps'],
    output_dir=model_dir_path,
    load_best_model_at_end=True,
    greater_is_better=False,
    metric_for_best_model="eval_loss",
)

# 学習データをバッチ処理するための設定
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Trainerの初期化
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    args=training_args,
    data_collator=data_collator,
)

# 学習を実行
trainer.train()

# outputフォルダに学習後のモデルを保存
model.save_pretrained(model_dir_path)

/home/saito/workspace/myenv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

### 学習したモデルのテスト
学習が完了していないので現状の出力はあまり良いものではありません。  

In [106]:
# テスト用のデータを取得
test_data = tokenized_valid_dataset[0]

print(f"[入力データ]\ninstruction:\n{test_data['instruction']}\ninput:\n{test_data['input']}\noutput:\n{test_data['output']}\n\n")

# テスト用の入力データを準備
test_input_ids = torch.tensor(test_data['input_ids']).unsqueeze(0).to(model.device)

# モデルを評価モードに設定
model.eval()

print(f"[出力]")
# 推論を実行
with torch.no_grad():
    test_output = model.generate(test_input_ids, **generate_config,streamer=streamer)

# 結果をデコードして表示
# print("出力:", tokenizer.decode(test_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[入力データ]
instruction:
RELXはどのような株価指数に属しているのですか？
input:
RELX plc（発音：レルエックス）は、英国ロンドンに本社を置く英国[2]の多国籍情報・分析企業です。科学・技術・医療情報および分析、法律情報および分析、意思決定ツールの提供、展示会の開催などの事業を展開しています。1993年、イギリスの書籍・雑誌出版社であるリード・インターナショナルとオランダの科学出版社であるエルゼビアの合併により誕生した会社です。

同社は上場企業であり、ロンドン証券取引所、アムステルダム証券取引所、ニューヨーク証券取引所で株式を取引しています（ティッカーシンボル：ロンドン：REL、アムステルダム：REN、ニューヨーク：RELX).FTSE100指数、Financial Times Global 500、Euronext 100指数の構成銘柄の一つです。
output:
RELX plcは、FTSE 100、Financial Times Global 500、Euronext 100の各インデックスを構成しています。


[出力]


RELXは、科学・技術および医療情報、法律情報、意思決定支援ツールの3つの主要事業分野をカバーする多国籍企業です。

科学・技術情報は、科学・テクノロジー・医学の研究と開発に関連
